In [1]:
# Importation des modules

import pandas as pd
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
#Importation des données

df = pd.read_csv('data/bdd_complete2.csv', sep = ',', encoding = 'latin-1')

In [31]:
# Echantillon de la base
print(df.shape)
df.sample(5)

(499843, 26)


,Unnamed: 0,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor
493905,493906,xavier batut,2017-10-10,Questions au Gouvernement,Présidence de M. François de Rugy,Couverture des territoires ruraux en téléphoni...,accéder au réseau EDGE relevait alors souven...,NaN,Présidence de M. François de Rugy,Batut,M,41,LREM,Defense,0,False,0,1 - Soc. civile,1 - Novice,PAS D'INFO,1976,True,cadsupprive,493906,Novice,2017-10-01
31919,31920,aurore berge,2019-07-02,Conservation et restauration de Notre-Dame de ...,Présidence de M. Richard Ferrand,Discussion générale,Pour respecter la volonté des donateurs nous ...,NaN,Présidence de M. Richard Ferrand,Bergé,F,31,LREM,Aff. Cult. & Educ.,0,False,3,4 - Entourage,4 - Entourage,ScPo,1986,True,collab,31920,Exp,2019-07-01
475782,475783,valerie bazin malgras,2018-05-29,Équilibre dans le secteur agricole et alimentaire,Présidence de M. François de Rugy,Discussion des articles,De plus ce dispositif est contre-productif. E...,NaN,Présidence de M. François de Rugy,Bazin-Malgras,F,48,LR,Aff. Cult. & Educ.,0,False,3,1 - Soc. civile,6  Exp. Pol.,PAS D'INFO,1969,False,EmployeInter,475783,Exp,2018-05-01
305135,305136,ludovic pajot,2017-12-12,Orientation et réussite des étudiants,Présidence de M. Marc Le Fur,Discussion des articles,Au-delà de ces aspects techniques larticle 1,NaN,Présidence de M. Marc Le Fur,Pajot,M,24,NI,DvptDurable,0,False,0,1 - Soc. civile,6  Exp. Pol.,PAS D'INFO,1993,False,NaN,305136,Exp,2017-12-01
297305,297306,laurent pietraszewski,2017-11-23,Renforcement du dialogue social,Présidence de M. Sylvain Waserman,Discussion des articles,Il est cohérent de soumettre le licenciement é...,NaN,Présidence de M. Sylvain Waserman,Pietraszewski,M,51,LREM,Aff. Sociales,0,False,0,1 - Soc. civile,1 - Novice,Master,1966,True,cadsupprive,297306,Novice,2017-11-01


In [32]:
# L'objectif est de nettoyer le texte dit par les intervenants. On s'intéresse donc aux colonnes 'Réplique' et 'Didascalie'

df_texte = df[['Réplique', 'Didascalie']]

In [33]:
df_texte.sample(10)

,Réplique,Didascalie
433709,au burn-out dans vos rangs. Surtout nous ri...,NaN
65335,LAssemblée parlementaire franco-allemande si...,NaN
61806,rapporteure. Défavorable.,NaN
125179,Cest avec attention et pragmatisme que jécou...,NaN
90774,et que cest en son sein en respectant les ...,NaN
85781,La commission a émis un avis favorable à cet a...,NaN
349979,rapporteur. À la place du nécessaire débat de ...,(Applaudissements sur quelques bancs du group...
267158,Je fais référence à larticle 50 du règlement...,NaN
272038,Cest en effet le problème<U+0080>!,NaN
313107,Quelques éléments nous sont parvenus sur ce su...,NaN


In [34]:
# Première correction : les apostrophes sont codées par '\x92'. On les retire. 
# On supprime aussi les codes '<U+0080>' qui sont avant les points d'exclamation
# On remplace les tirets par des espaces
# On met tout en minuscule

df_texte['Réplique'] = df_texte.apply(
    lambda row: row.Réplique.replace('\x92', '').replace('<U+0080>', '').replace('-', ' ').lower(),
    axis = 1)

df_texte['Didascalie'] = df_texte.apply(
    lambda row: str(row.Didascalie).replace('\x92', '').replace('<U+0080>', '').replace('-', ' ').lower(),
    axis = 1)


/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [35]:
# Séparation des didascalies

import re

def extract_didascalie(str):
    li=[]
    result = re.search(r'\((.*?)\)',str)
    if result !=None:
        n=len(result.groups())
        for i in range(1,n+1):
            a=''.join(result.groups(i))
            a=a.replace('(','')
            a=a.replace(',','')
            a=a.replace(')','')
            a=a.replace("'",'')
            li.append(a)
    return li

def extract_replique(str):
    rep=str
    li=extract_didascalie(str)
    for i in li:
        rep=rep.replace(i,"")
        
    rep=rep.replace("()","")   
    return rep

df_texte['Réplique'] = df_texte.apply(lambda row: extract_replique(row.Réplique), axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
# On tokenize les chaines de caractère

df_texte['tokenized_replique'] = df_texte.apply(lambda row: nltk.word_tokenize(row.Réplique, language='french'), 
                                             axis = 1)
df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: nltk.word_tokenize(row.Didascalie, language='french'), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
# On obtient bien des listes de mots

df_texte.head(10)

# On remplace les nan par des listes vides

df_texte['tokenized_replique'] = df_texte.apply(lambda row: row.tokenized_replique if len(row.tokenized_replique) == 0 or row.tokenized_replique[0] != 'nan' else [], axis = 1)
df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: row.tokenized_didascalie if len(row.tokenized_didascalie) == 0 or row.tokenized_didascalie[0] != 'nan' else [], axis = 1)



/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
# On retire la ponctuation pour le moment

df_texte['tokenized_replique'] = df_texte.apply(lambda row: [word for word in row.tokenized_replique if word.isalpha()], 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: [word for word in row.tokenized_didascalie if word.isalpha()], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [39]:
# On retire les stopwords

stop_words = set(stopwords.words('french'))

df_texte['tokenized_replique'] = df_texte.apply(lambda row: [word for word in row.tokenized_replique if not word in stop_words], 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: [word for word in row.tokenized_didascalie if not word in stop_words], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [46]:
# Stemming

df_texte['stemmed_replique'] = df_texte.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.tokenized_replique]), 
                                             axis = 1)

df_texte['stemmed_didascalie'] = df_texte.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.tokenized_didascalie]), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [48]:
# On repasse les listes en chaines de caractère


df_texte['tokenized_replique'] = df_texte.apply(lambda row: ' '.join(row.tokenized_replique), 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: ' '.join(row.tokenized_didascalie), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
df_cleaned = df.copy()
df_cleaned['tokenized_replique'] = df_texte['tokenized_replique']
df_cleaned['tokenized_didascalie'] = df_texte['tokenized_didascalie']
df_cleaned['stemmed_replique'] = df_texte['stemmed_replique']
df_cleaned['stemmed_didascalie'] = df_texte['stemmed_didascalie']

In [ ]:
# df_cleaned.to_csv('data/data_cleaned.csv', sep = ',', encoding = 'latin-1')

In [2]:
pd.read_csv('data/data_cleaned.csv', sep = ',', encoding = 'latin-1').sample(10)

,Unnamed: 0,Unnamed: 0.1,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor,tokenized_replique,tokenized_didascalie,stemmed_replique,stemmed_didascalie
294196,294196,294197,laurent furst,2018-02-14,Élection des représentants au Parlement européen,Présidence de M. François de Rugy,Discussion des articles,Jaimerais dire à M. Mendes que jai adoré sa ...,(Protestations sur les bancs du groupe REM.),Présidence de M. François de Rugy,Furst,M,52,LR,Defense,1,False,22,5 - Dép. sortant,5 - Dép. sortant,ScPoProv,1965,False,cadsuppub,294197,Exp,2018-02-01,jaimerais dire mendes jai adoré leçon morale m...,protestations bancs groupe rem,jaim dir mend jai ador leçon moral major nouve...,protest banc group rem
233100,233100,233101,jean felix acquaviva,2018-07-17,Démocratie plus représentative responsable et...,Présidence de M. François de Rugy,Discussion des articles,Dune part il est possible de faire en sorte ...,NaN,Présidence de M. François de Rugy,Acquaviva,M,44,NI,Aff. Cult. & Educ.,0,False,9,2 - 2de carrière,2 - 2de carrière,Master,1973,False,cadsupprive,233101,Exp,2018-07-01,dune part possible faire sorte sénat travaille...,NaN,dun part possibl fair sort sénat travaill deve...,NaN
410007,410007,410008,richard ferrand,2020-03-04,Système universel de retraite,Présidence de M. Hugues Renson,Discussion des articles,La parole est à M. Pierre Dharréville pour s...,NaN,Présidence de M. Hugues Renson,Ferrand,M,55,LREM,Defense,1,True,22,5 - Dép. sortant,5 - Dép. sortant,Master,1962,True,cadsupprive,410008,Exp,2020-03-01,parole pierre dharréville soutenir sous amende...,NaN,parol pierr dharrévill souten sous amend,NaN
480114,480114,480115,valerie rabault,2018-05-24,Équilibre dans le secteur agricole et alimentaire,Présidence de M. de Rugy,Discussion des articles,Cet article 4 sonne pour vous lheure de vérit...,NaN,Présidence de M. de Rugy,Rabault,F,44,NG,Finances,1,False,5,5 - Dép. sortant,5 - Dép. sortant,GdEc,1973,False,cadsupprive,480115,Exp,2018-05-01,cet article sonne lheure vérité deçà annoncé e...,NaN,cet articl son lheur vérit deçà annonc engag p...,NaN
222084,222084,222085,ian boucard,2020-02-13,Financement des infrastructures de téléphonie ...,Présidence de M. Sylvain Waserman,Discussion générale,Le ministère de la cohésion des territoires e...,NaN,Présidence de M. Sylvain Waserman,Boucard,M,29,LR,Defense,0,True,4,4 - Entourage,4 - Entourage,PtEc,1988,False,cadsupprive,222085,Exp,2020-02-01,ministère cohésion territoires relations colle...,NaN,minister cohes territoir relat collect territo...,NaN
345018,345018,345019,nadia hai,2019-12-16,Projet de loi de finances pour 2020,Présidence de Mme Laetitia Saint-Paul,Discussion générale,Le projet de loi de finances pour 2020 dont ...,NaN,Présidence de Mme Laetitia Saint-Paul,Hai,F,37,LREM,Finances,0,False,0,1 - Soc. civile,1 - Novice,Licence,1980,True,EmployeInter,345019,Novice,2019-12-01,projet loi finances dont entamons nouvelle lec...,NaN,projet loi financ dont entamon nouvel lectur t...,NaN
37074,37074,37075,beatrice piron,2018-06-06,Évolution du logement de laménagement et du ...,Présidence de M. François de Rugy,Discussion des articles,Je le maintiens.,NaN,Présidence de M. François de Rugy,Piron,F,52,LREM,Aff. Cult. & Educ.,0,False,0,1 - Soc. civile,1 - Novice,GdEc,1965,True,AgriArtiCommPDG,37075,Novice,2018-06-01,maintiens,NaN,maintien,NaN
252733,252733,252734,jean marie sermier,2020-02-18,Questions au Gouvernement,Présidence de M. Richard Ferrand,Centrale nucléaire de Fessenheim,Ce nest pas vrai et vous le savez bien<U+00...,NaN,Présidence de M. Richard Ferrand,Sermier,M,56,LR,DvptDurable,3,False,31,5 - Dép. sortant,5 - Dép. sortant,Pro,1961,False,AgriArtiCommPDG,252734,Exp,2020-02-01,nest vrai savez bien,NaN,nest vrai sav bien,NaN
211347,211347,211348,gilles lurton,2